In [2]:
from torchvision import datasets
import torch 
data_folder = './data/'
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)




In [3]:
tr_images = fmnist.data 
tr_targets = fmnist.targets
print(len(tr_images), len(tr_targets))

60000 60000


In [4]:
val_fmnist = datasets.FashionMNIST(data_folder, download=True, train=False)
val_images = val_fmnist.data 
val_targets = val_fmnist.targets 
print(len(val_images), len(val_targets))


10000 10000


In [5]:
import matplotlib.pyplot as plt 
%matplotlib inline 
import numpy as np 
from torch.utils.data import Dataset, DataLoader 
import torch 
import torch.nn as nn 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cuda


In [6]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()/255
        x = x.view(-1,1,28,28)
        self.x, self.y = x, y 
    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]        
        return x.to(device), y.to(device)
    def __len__(self): 
        return len(self.x)

from torch.optim import SGD, Adam
def get_model():
    model = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(3200, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

def train_batch(x, y, model, opt, loss_fn):
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()


In [7]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=2, shuffle=True)
    val = FMNISTDataset(val_images, val_targets)
    val_dl = DataLoader(val, batch_size=2, shuffle=True)
    return trn_dl, val_dl

In [8]:
@torch.no_grad()
def val_loss(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [9]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()

In [10]:
from torchsummary import summary
model, loss_fn, optimizer = get_model()
summary(model, (1, 28, 28))


RuntimeError: mat1 dim 1 must match mat2 dim 0